# Picasso Demo

## Reference (Picasso):
* https://wiki.openstack.org/wiki/Picasso
* https://github.com/openstack/picasso/blob/master/devstack/README.md

## Reference (Iron.io):
* https://github.com/iron-io/functions

## Prerequisite:
- Vagrant
- devstack

Example configuration of Vagrantfile:

```:Vagrantfile
Vagrant.configure("2") do |config|
  config.vm.box = "ubuntu/xenial64"

  # config.vm.box_check_update = false

  config.vm.network "forwarded_port", guest: 80, host: 8080
  config.vm.network "forwarded_port", guest: 8888, host: 18888

  config.vm.network "private_network", ip: "192.168.33.10"

  config.vm.provider "virtualbox" do |vb|
  #   # Display the VirtualBox GUI when booting the machine
  #   vb.gui = true
  #
  #   # Customize the amount of memory on the VM:
    vb.memory = "8192"
  end
end
```

### Install glide, devstack and picasso

Download devstack.

```
sudo yum install -y git glide
sudo env GOPATH=/usr/local/go/bin go get -u github.com/golang/dep/...
export DEVSTACK_DIR=~/devstack
cd ~
git clone git://git.openstack.org/openstack-dev/devstack.git $DEVSTACK_DIR
cd devstack
cp samples/local.conf ./
```

As a next step, edit local.conf as follows.

Note: You have to enable aodh and ceilometer to confirm alarm notification.

```
HOST_IP=192.168.33.10

enable_plugin aodh https://git.openstack.org/openstack/aodh
enable_plugin picasso https://github.com/openstack/picasso.git

CEILOMETER_BACKEND=mongodb
enable_plugin ceilometer https://git.openstack.org/openstack/ceilometer

# Picasso configuration
PICASSO_REPO=${PICASSO_REPO:-https://github.com/openstack/picasso.git}
PICASSO_BRANCH=${PICASSO_BRANCH:-master}
PICASSO_DIR=${PICASSO_DIR:-${DEST}/picasso}
PICASSO_PORT=${PICASSO_PORT:-10001}
PICASSO_LOG_LEVEL=${PICASSO_LOG_LEVEL:-DEBUG}
PICASSO_LOG_FILE=${PICASSO_LOG_FILE:-/var/log/picasso-api.log}

# Picasso client configuration
PICASSO_CLIENT_REPO=${PICASSO_CLIENT_REPO:-https://github.com/openstack/python-picassoclient.git}
PICASSO_CLIENT_DIR=${PICASSO_CLIENT_DIR:-${DEST}/python-picassoclient}
PICASSO_CLIENT_BRANCH=${PICASSO_CLIENT_BRANCH:-master}

# Functions parameters
FUNCTIONS_REPO=${FUNCTIONS_REPO:-https://github.com/iron-io/functions.git}
FUNCTIONS_BRANCH=${FUNCTIONS_BRANCH:-master}
FUNCTIONS_PORT=${FUNCTIONS_PORT:-10501}
FUNCTIONS_DB=${FUNCTIONS_DBPATH:-bolt://$FUNCTIONS_DIR/devstack.functions.storage.db?bucket=funcs}
FUNCTIONS_MQ=${FUNCTIONS_DBPATH:-bolt://$FUNCTIONS_DIR/devstack.functions.queue.db}
FUNCTIONS_LOG_LEVEL=${FUNCTIONS_LOG_LEVEL:-DEBUG}

DOCKERD_OPTS=${DOCKERD_OPTS:---dns 8.8.8.8 --dns 8.8.4.4 --storage-driver=devicemapper -H fd://}
```

Then, invoke stack.sh

```
./stack.sh
```

## Known Issues
In CentOS7.2 environment, some of Docker options are not supported for now.
You might be change a systemd configuration (/usr/lib/systemd/system/docker.service) as follows.

```
[Unit]
Description=Docker Application Container Engine
Documentation=https://docs.docker.com
After=network.target docker.socket
#Requires=docker.socket

[Service]
Type=notify
Environment=SERVICE_HOST=127.0.0.1
ExecStart=/usr/bin/dockerd -H 0.0.0.0:2375 --dns 8.8.8.8 --dns 8.8.4.4 --storage-driver=devicemapper
MountFlags=slave
LimitNOFILE=1048576
LimitNPROC=1048576
LimitCORE=infinity
TimeoutStartSec=0
Delegate=yes

[Install]
WantedBy=multi-user.target
```

In [1]:
source ~/devstack/openrc

In [2]:
openstack fn apps list

In [3]:
openstack fn apps create alarm-notifier-app
func_name=`openstack fn apps list | grep alarm-notifier-app | awk '{print $2}'`
echo $func_name

+-------------+--------------------------------------------------+
| Field       | Value                                            |
+-------------+--------------------------------------------------+
| description | App for project 5811d3471a5742878fb7869bd57d4b9a |
| created_at  | 2017-03-09 03:25:02.737626                       |
| updated_at  | 2017-03-09 03:25:02.737652                       |
| project_id  | 5811d3471a5742878fb7869bd57d4b9a                 |
| config      | None                                             |
| id          | 6e034fea178c4cfaad511632b4a00f4c                 |
| name        | alarm-notifier-app-5811d3471a5                   |
+-------------+--------------------------------------------------+
alarm-notifier-app-5811d3471a5


In [4]:
openstack fn routes create $func_name /hello async iron/hello --is-public

+-----------------+------------+
| Field           | Value      |
+-----------------+------------+
| image           | iron/hello |
| memory          | 128        |
| max_concurrency | 1          |
| timeout         | 30         |
| path            | /hello     |
| is_public       | True       |
| type            | async      |
+-----------------+------------+


In [5]:
nova boot --image $(glance image-list | awk '/cirros-0.3.5-x86_64-disk / {print $2}') --flavor 42 test_alarms

/usr/lib/python2.7/site-packages/novaclient/client.py:278: UserWarning: The 'tenant_id' argument is deprecated in Ocata and its use may result in errors in future releases. As 'project_id' is provided, the 'tenant_id' argument will be ignored.
  warnings.warn(msg)
+--------------------------------------+-----------------------------------------------------------------+
| Property                             | Value                                                           |
+--------------------------------------+-----------------------------------------------------------------+
| OS-DCF:diskConfig                    | MANUAL                                                          |
| OS-EXT-AZ:availability_zone          |                                                                 |
| OS-EXT-STS:power_state               | 0                                                               |
| OS-EXT-STS:task_state                | scheduling                                          

In [6]:
aodh alarm create \
    --type threshold \
    --name cpu_high \
    --description 'instance running hot' \
    --meter-name cpu_util \
    --threshold 20.0 \
    --comparison-operator gt \
    --statistic max \
    --period 600 \
    --evaluation-periods 1 \
    --alarm-action $(openstack fn routes expose-url $func_name /hello) \
    --query resource_id=$(nova list | grep test_alarms | awk '{print $2}')

/usr/lib/python2.7/site-packages/novaclient/client.py:278: UserWarning: The 'tenant_id' argument is deprecated in Ocata and its use may result in errors in future releases. As 'project_id' is provided, the 'tenant_id' argument will be ignored.
  warnings.warn(msg)
+---------------------------+--------------------------------------------------+
| Field                     | Value                                            |
+---------------------------+--------------------------------------------------+
| alarm_actions             | [u'http://172.21.78.153:10001/r/alarm-notifier-  |
|                           | app-5811d3471a5/hello']                          |
| alarm_id                  | 4bb27260-613d-4075-b216-6eb28c1b6499             |
| comparison_operator       | gt                                               |
| description               | instance running hot                             |
| enabled                   | True                                             |
| eval

# Place a load on the instance

As a first step, log in to the instance.

Then, invoke dd command so that aodh alarm is going to be fired.

```
openstack security group rule create default --protocol icmp --remote-ip 0.0.0.0/0
openstack security group rule create default --protocol tcp --dst-port 22:22 --remote-ip 0.0.0.0/0

ns_rq=`ip netns show | grep qrouter`
sudo ip netns exec $ns_rq bash
ssh cirros@10.0.0.8
dd if=/dev/zero of=/dev/null
```

In [8]:
aodh alarm-history search

+-------------------+-------------------+------------------+-------------------+
| alarm_id          | timestamp         | type             | detail            |
+-------------------+-------------------+------------------+-------------------+
| 4bb27260-613d-407 | 2017-03-09T03:50: | state transition | {"transition_reas |
| 5-b216-6eb28c1b64 | 09.093737         |                  | on": "Transition  |
| 99                |                   |                  | to alarm due to 1 |
|                   |                   |                  | samples outside   |
|                   |                   |                  | threshold, most   |
|                   |                   |                  | recent:           |
|                   |                   |                  | 99.5579193086",   |
|                   |                   |                  | "state": "alarm"} |
| 4bb27260-613d-407 | 2017-03-09T03:26: | creation         | {"alarm_actions": |
| 5-b216-6eb28c1b64 | 13.345

## How to check log of execution

It seems that Iron.io does not support convenient way to check execution history so far.
You can track history on syslog.

In [9]:
sudo grep "Mar  9 12:50" /var/log/messages  | grep functions:

Mar  9 12:50:09 kvm526 functions: time="2017-03-09T12:50:09+09:00" level=debug msg="Finding route on datastore" action="server.handleRunnerRequest)-fm" app=alarm-notifier-app-5811d3471a5 call_id=910c28b4-1d3c-5c6e-b6c8-1b2d5a43df29 path="/hello" route="/hello"
Mar  9 12:50:09 kvm526 functions: time="2017-03-09T12:50:09+09:00" level=debug msg="Got routes from datastore" action="server.handleRunnerRequest)-fm" app=alarm-notifier-app-5811d3471a5 call_id=910c28b4-1d3c-5c6e-b6c8-1b2d5a43df29 route="/hello" routes=1
Mar  9 12:50:09 kvm526 functions: time="2017-03-09T12:50:09+09:00" level=info msg="Pushed to MQ" call_id=910c28b4-1d3c-5c6e-b6c8-1b2d5a43df29
Mar  9 12:50:09 kvm526 functions: time="2017-03-09T12:50:09+09:00" level=info msg="Added new task to queue" action="server.handleRunnerRequest)-fm" app=alarm-notifier-app-5811d3471a5 call_id=910c28b4-1d3c-5c6e-b6c8-1b2d5a43df29 image="iron/hello" route="/hello"
Mar  9 12:50:09 kvm526 functions: time="2017-03-09T12:50:09+09:00" level=info ms

## How to display apps / routes by using openstack CLI

In [10]:
openstack fn routes list $func_name

+-------+--------+----------+--------+---------+-----------------+-----------+--------+
| type  | path   | image    | memory | timeout | max_concurrency | is_public | config |
+-------+--------+----------+--------+---------+-----------------+-----------+--------+
| async | /hello | iron/hel |    128 |      30 |               1 | True      |        |
|       |        | lo       |        |         |                 |           |        |
+-------+--------+----------+--------+---------+-----------------+-----------+--------+


In [11]:
openstack fn routes show $func_name /hello

+-----------------+------------+
| Field           | Value      |
+-----------------+------------+
| image           | iron/hello |
| memory          | 128        |
| max_concurrency | 1          |
| timeout         | 30         |
| path            | /hello     |
| is_public       | True       |
| type            | async      |
+-----------------+------------+


## How to display apps / routes by using iron.io CLI

Operators can also be accessd to iron.io via native API or fn-CLI directly.

To install 'fn' command, invoke following url.
```
curl -LSs https://goo.gl/VZrL8t | sh
```

In [12]:
export API_URL=http://localhost:10501

In [13]:
fn apps list

alarm-notifier-app-5811d3471a5
alarm-notifier-app-db507b128d5


In [14]:
fn routes list $func_name

path	image	endpoint
/hello	iron/hello


In [15]:
fn routes call $func_name /hello

{"call_id":"687e0b49-c8b0-5343-b6dd-c795494810d9"}


In [16]:
sudo grep 687e0b49-c8b0-5343-b6dd-c795494810d9 /var/log/messages 

Mar  9 12:58:53 kvm526 functions: time="2017-03-09T12:58:53+09:00" level=debug msg="Finding route on datastore" action="server.handleRunnerRequest)-fm" app=alarm-notifier-app-5811d3471a5 call_id=687e0b49-c8b0-5343-b6dd-c795494810d9 path="/hello" route="/hello"
Mar  9 12:58:53 kvm526 functions: time="2017-03-09T12:58:53+09:00" level=debug msg="Got routes from datastore" action="server.handleRunnerRequest)-fm" app=alarm-notifier-app-5811d3471a5 call_id=687e0b49-c8b0-5343-b6dd-c795494810d9 route="/hello" routes=1
Mar  9 12:58:53 kvm526 functions: time="2017-03-09T12:58:53+09:00" level=info msg="Pushed to MQ" call_id=687e0b49-c8b0-5343-b6dd-c795494810d9
Mar  9 12:58:53 kvm526 functions: time="2017-03-09T12:58:53+09:00" level=info msg="Added new task to queue" action="server.handleRunnerRequest)-fm" app=alarm-notifier-app-5811d3471a5 call_id=687e0b49-c8b0-5343-b6dd-c795494810d9 image="iron/hello" route="/hello"
Mar  9 12:58:54 kvm526 functions: time="2017-03-09T12:58:54+09:00" level=info ms

## How to create functions

```
export DOCKER_HOST=tcp://localhost:2375

mkdir ~/picasso-func ; cd ~/picasso-func
vi func.go
```

func.go:
```
package main

import (
    "encoding/json"
    "fmt"
    "os"
)

type Person struct {
    Name string
}

func main() {
    p := &Person{Name: "World"}
    json.NewDecoder(os.Stdin).Decode(p)
    fmt.Printf("Hello %v!", p.Name)
}
```

```
fn init testuser/hello
fn build
fn run
fn build && fn push
fn apps create myapp
fn routes create myapp /hello
```

In [18]:
export DOCKER_HOST=tcp://localhost:2375
docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
testuser/hello      0.0.1               d2a63df5a52d        5 minutes ago       12.6 MB
picasso-api         latest              f8aa15239468        2 hours ago         791 MB
<none>              <none>              60be38a5d6e7        20 hours ago        791 MB
<none>              <none>              dbf4f0595bb5        24 hours ago        791 MB
<none>              <none>              42a97c056272        26 hours ago        791 MB
<none>              <none>              d719aa45609a        26 hours ago        791 MB
python              3.5                 4e5ed9f6613e        8 days ago          687 MB
iron/go             dev                 3d25645a73f8        2 weeks ago         857 MB
iron/hello          latest              f4dd2a73dac8        4 months ago        12.1 MB
iron/go             latest              c05f82fa066a        12 months ago       10.5 MB


In [19]:
curl $API_URL/r/myapp/hello

Hello World!

In [20]:
fn apps list

alarm-notifier-app-5811d3471a5
alarm-notifier-app-db507b128d5
myapp


In [21]:
fn routes  list myapp

path	image	endpoint
/hello	testuser/hello:0.0.1
